In [1]:
#import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gmaps

from config import census_key,g_key

gmaps.configure(g_key)

In [3]:
data = pd.read_csv("../Output/Data/census_data.csv")
byzipcode = data.copy()
races = byzipcode.columns[4:]
for race in races:
    byzipcode[f'{race}%'] = byzipcode[race]/byzipcode['population']

lat_log = pd.read_csv('../Resources/us-zip-code-latitude-and-longitude.csv', sep = ';')
lat_log = lat_log[['Zip','geopoint']]

byzipcode = pd.merge(lat_log,byzipcode, how = 'right', right_on = 'zipcode', left_on = 'Zip')
byzipcode.drop(columns= 'zipcode', inplace = True)
byzipcode.rename(columns = {'Zip':'zipcode'},inplace = True)
byzipcode.head()

,zipcode,geopoint,city,population,per_capita_income,white,african_american,native_american,asian,native_hawaiian_pacific_islander,...,hispanic_or_latino,white%,african_american%,native_american%,asian%,native_hawaiian_pacific_islander%,other_race%,two_or_more_races%,white_not_hispanic%,hispanic_or_latino%
0,46214,"39.792993,-86.28575",Indianapolis,24202.0,26462.0,12760.0,8746.0,30.0,446.0,189.0,...,3312.0,0.527229,0.361375,0.001240,0.018428,0.007809,0.035865,0.048054,0.450459,0.136848
1,75087,"32.93382,-96.45446",Dallas,33638.0,39436.0,30220.0,954.0,90.0,873.0,0.0,...,4076.0,0.898389,0.028361,0.002676,0.025953,0.000000,0.017926,0.026696,0.797342,0.121172
2,30336,"33.744007,-84.56032",Atlanta,234.0,42524.0,33.0,186.0,0.0,7.0,0.0,...,11.0,0.141026,0.794872,0.000000,0.029915,0.000000,0.000000,0.034188,0.123932,0.047009
3,19104,"39.961612,-75.19957",Philadelphia,54311.0,15478.0,20419.0,22962.0,291.0,7662.0,62.0,...,2766.0,0.375964,0.422787,0.005358,0.141076,0.001142,0.015430,0.038243,0.345823,0.050929
4,19146,"39.938512,-75.18067",Philadelphia,38873.0,45507.0,18549.0,16645.0,120.0,1701.0,35.0,...,1799.0,0.477169,0.428189,0.003087,0.043758,0.000900,0.015023,0.031873,0.446042,0.046279


In [4]:
seattle_df = pd.read_csv("../Output/Data/seattle_restaurants.csv")
phila_df = pd.read_csv("../Output/Data/philadelphia_restaurants.csv")
dallas_df = pd.read_csv("../Output/Data/dallas_restaurants.csv")
indie_df = pd.read_csv("../Output/Data/indianapolis_restaurants.csv")
alt_df = pd.read_csv("../Output/Data/atlanta_restaurants.csv")

yelp_df = pd.concat([seattle_df,phila_df,dallas_df,indie_df,alt_df])
yelp_df = yelp_df.reset_index()
yelp_df.drop(columns = ['index','Unnamed: 0'], inplace = True)
yelp_df

FileNotFoundError: [Errno 2] File ../Output/Data/seattle_restaurants.csv does not exist: '../Output/Data/seattle_restaurants.csv'

In [ ]:
df = pd.merge(yelp_df,byzipcode, how = 'left',on = 'zipcode')
df

In [ ]:
#zipcodes where there is no census data or is not actually part of the city
df[df['city_y'].isnull()]['zipcode'].unique()


In [ ]:
#drop these rows. can't use simple drop na as we want to keep NaA in price
df.drop(index = df[df['city_y'].isnull()].index, inplace = True)
df.drop(columns = ['city_y'], inplace = True)
df.rename(columns = {'name' : 'restaurant_name',
                     'city_x' : 'city', 
                     'geopoint': 'zipcode geopoint', 
                     'ethnicity':'restaurant_ethnicity'}, inplace = True)

In [ ]:
df.columns

In [ ]:
#want to reorder columns
df = df[['restaurant_name','restaurant_ethnicity','latitude', 'longitude',
        'zipcode','zipcode geopoint','city','price', 'rating', 
       'population', 'per_capita_income', 'white', 'african_american',
       'native_american', 'asian', 'native_hawaiian_pacific_islander',
       'other_race', 'two_or_more_races', 'white_not_hispanic',
       'hispanic_or_latino', 'white%', 'african_american%', 'native_american%',
       'asian%', 'native_hawaiian_pacific_islander%', 'other_race%',
       'two_or_more_races%', 'white_not_hispanic%', 'hispanic_or_latino%']]